### MobileNet Model Training Emotion Detection

In [8]:
import os
import shutil
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import tensorflow as tf
from keras.utils import load_img
from keras.preprocessing.image import ImageDataGenerator

from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import nbformat

import warnings
warnings.filterwarnings("ignore")

In [9]:
class CFG:
    batch_size = 64
    img_height = 48
    img_width = 48
    epoch = 25

In [10]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [11]:
# Get Number of Images for each Emotion and Splitting

train_path = "Data_E/train"
test_path = "Data_E/test"
emotions = os.listdir(train_path)

len_train = {}
len_public_test = {}
len_private_test = {}

for emotion in emotions:
    file_train = os.listdir(os.path.join(train_path, emotion))
    len_train[emotion] = len(file_train)
    
    file_test = os.listdir(os.path.join(test_path, emotion))
    file_public_test = [f for f in file_test if "Public" in f]
    file_private_test = [f for f in file_test if "Private" in f]
    len_public_test[emotion] = len(file_public_test)
    len_private_test[emotion] = len(file_private_test)

print("Train: ", len_train)
print("Public Test: ", len_public_test)
print("Private Test: ", len_private_test)

total_train = sum(len_train.values())
total_public_test = sum(len_public_test.values())
total_private_test = sum(len_private_test.values())

print("Total Train: ", total_train)
print("Total Public Test: ", total_public_test)
print("Total Private Test: ", total_private_test)

FileNotFoundError: [Errno 2] No such file or directory: 'Data_E/train'

In [17]:
# Class Distribution

# Create Subplots
data_split = ["Train", "Public Test", "Private Test"]
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=data_split
)

all_len = [len_train, len_public_test, len_private_test]
all_total = [total_train, total_public_test, total_private_test]

for i in range(3):
    # Bar Chart
    fig.add_trace(
        go.Bar(
            x=list(all_len[i].keys()),
            y=list(all_len[i].values()),
            name=data_split[i],
            marker_color=["#935d39", "#926f48", "#90825c", "#8e9590", "#8ba0a5", "#88acb9", "#85b8cd"],
        ), row=i+1, col=1
    )

    # Update Axes
    fig.update_xaxes(ticks="outside", linecolor="Black", row=i+1, col=1)
    fig.update_yaxes(ticks="outside", linecolor="Black", row=i+1, col=1)

# Update Layout
fig.update_layout(
    title="Class Distribution", title_x=0.5, font_size=14, font_family="Cambria",
    width=650, height=900,
    plot_bgcolor="White",
    showlegend=False
)

# Show
# fig.show(renderer="iframe_connected")

NameError: name 'len_train' is not defined

In [18]:
# Data Augmentation
def data_augmentation():
    # Training Dataset
    TRAINING_DIR = "Data_E/train"
    train_datagen = ImageDataGenerator(
        rescale=1/255.,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2
    )
    train_generator = train_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size=(CFG.img_height, CFG.img_width),
        batch_size=CFG.batch_size,
        class_mode="categorical",
        shuffle=True,
        subset="training"
    )

    # Validation Dataset
    validation_generator = train_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size=(CFG.img_height, CFG.img_width),
        batch_size=CFG.batch_size,
        class_mode="categorical",
        shuffle=False,
        subset="validation"
    )

    # Public Testing Dataset
    PUBLIC_TESTING_DIR = "public_test"
    public_test_datagen = ImageDataGenerator(rescale=1./255.)
    public_test_generator = public_test_datagen.flow_from_directory(
        PUBLIC_TESTING_DIR,
        target_size=(CFG.img_height, CFG.img_width),
        batch_size=1,
        class_mode="categorical",
        shuffle=False
    )
    
    # Private Testing Dataset
    PRIVATE_TESTING_DIR = "private_test"
    private_test_datagen = ImageDataGenerator(rescale=1./255.)
    private_test_generator = private_test_datagen.flow_from_directory(
        PRIVATE_TESTING_DIR,
        target_size=(CFG.img_height, CFG.img_width),
        batch_size=1,
        class_mode="categorical",
        shuffle=False
    )
    
    return train_generator, validation_generator, public_test_generator, private_test_generator

In [19]:
# Create data augmentation
seed_everything(2023)
train_generator, validation_generator, public_test_generator, private_test_generator = data_augmentation()

# Load the MobileNet model without the top layer
base_model = MobileNet(
    input_shape=(CFG.img_height, CFG.img_width, 3),
    include_top=False,
    weights='imagenet'
)

# Add a GlobalAveragePooling2D layer and a Dense layer with softmax activation for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Combine the base model with the new layers to create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the base model to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-5), metrics=['accuracy'])
model.summary()

FileNotFoundError: [Errno 2] No such file or directory: 'Data_E/train'

In [15]:
# Create Callback
model_checkpoint = ModelCheckpoint(
    "MobileNetV1.h5", 
    monitor="val_loss", 
    mode="min", 
    verbose=1,
    save_best_only=True
)

callbacks = [model_checkpoint]

#### Training and testing the data using MobileNet

In [16]:
# Train the model on the training data
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // CFG.batch_size,
    epochs=CFG.epoch,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // CFG.batch_size,
    callbacks=callbacks,
)

NameError: name 'model' is not defined

#### MobileNet Results

In [ ]:
# Evaluate on Public Test Data
public_scores = model.evaluate(public_test_generator)
print("%s: %.2f%%" % ("Evaluate Public Test Accuracy", public_scores[1]*100))

# Evaluate on Private Test Data
private_scores = model.evaluate(private_test_generator)
print("%s: %.2f%%" % ("Evaluate Private Test Accuracy", private_scores[1]*100))

3589/3589 [==============================] - 50s 14ms/step - loss: 2.3552 - accuracy: 0.1432
Evaluate Public Test Accuracy: 14.32%
3589/3589 [==============================] - 51s 14ms/step - loss: 2.3275 - accuracy: 0.1480
Evaluate Private Test Accuracy: 14.80%


In [ ]:
# Visualize Training and Validation Results

# Create Subplot
fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=["Model Loss", "Model Accuracy"], 
)

# Loss Plot
loss = history.history['loss']
val_loss = history.history['val_loss']
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(loss)+1), y=loss,
        mode="markers+lines",
        marker=dict(
            color="#935d39", size=6,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color="#935d39", width=1.5),
        name="Training Loss"
    ), row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(val_loss)+1), y=val_loss,
        mode="markers+lines",
        marker=dict(
            color="#85b8cd", size=6,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color="#85b8cd", width=1.5),
        name="Validation Loss"
    ), row=1, col=1
)

# Accuracy Plot
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(acc)+1), y=acc,
        mode="markers+lines",
        marker=dict(
            color="#935d39", size=6,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color="#935d39", width=1.5),
        name="Training Accuracy"
    ), row=1, col=2
)
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(val_acc)+1), y=val_acc,
        mode="markers+lines",
        marker=dict(
            color="#85b8cd", size=6,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color="#85b8cd", width=1.5),
        name="Validation Accuracy"
    ), row=1, col=2
)

# Update Axes
fig.update_xaxes(title="Epochs", linecolor="Black", ticks="outside", row=1, col=1)
fig.update_xaxes(title="Epochs", linecolor="Black", ticks="outside", row=1, col=2)
fig.update_yaxes(title="Categorical Loss", linecolor="Black", ticks="outside", row=1, col=1)
fig.update_yaxes(title="Accuracy", linecolor="Black", ticks="outside", row=1, col=2)

# Update Layout
fig.update_layout(
    title="Training and Validation Results", title_x=0.5, font_family="Cambria",
    width=950, height=400,
    showlegend=False,
    plot_bgcolor="White",
    paper_bgcolor="White"
)

# Show
fig.show(iframe_connected=True)

In [ ]:
# Confusion Matrix
predictions = model.predict(public_test_generator)

# Get the true labels from the generator
true_labels = public_test_generator.classes

# Compute the confusion matrix using tf.math.confusion_matrix
confusion_matrix = tf.math.confusion_matrix(
        labels=true_labels,
        predictions=predictions.argmax(axis=1),
        num_classes=7)

# Print the confusion matrix
print(confusion_matrix)